In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import os


data_file_name = 'simple_28features_30sec'

WORKING_DIR = os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir))
DATA_ROOT_DIR = os.path.join(WORKING_DIR, 'data')
DATA_MUSIC_DIR = os.path.join(DATA_ROOT_DIR, 'genres_original')

In [2]:
# cargamos los datos en un data frame
data = pd.read_csv(f'{DATA_ROOT_DIR}/{data_file_name}.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,tempo,mfcc1,mfcc2,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,classical.00074.wav,0.237426,0.037299,1359.905307,1572.452917,2568.617801,0.072117,161.499023,-289.79858,143.909320,...,-1.684359,3.078970,2.686264,2.535048,4.614130,2.376674,-1.273050,-3.541758,-2.806386,classical
1,classical.00021.wav,0.266760,0.045469,1371.283566,1440.791445,2570.001126,0.079567,135.999178,-278.21188,147.887740,...,-0.964953,-2.221944,-0.574204,1.875123,5.149712,2.630447,0.821938,-1.647068,-0.104045,classical
2,classical.00017.wav,0.256395,0.037378,1358.897903,1417.887433,2417.593714,0.081116,89.102909,-311.53937,148.683270,...,6.745241,4.605570,3.764831,1.275329,1.243601,-0.089599,1.511763,0.387477,-3.003955,classical
3,classical.00036.wav,0.239282,0.016010,1281.770846,1502.895726,2440.360243,0.059410,107.666016,-419.22890,127.448350,...,-5.439006,-9.099890,-10.616878,-5.737675,-0.392486,7.027566,5.593974,-3.469120,-5.166212,classical
4,classical.00097.wav,0.222324,0.036727,1494.115744,1670.924448,2780.841518,0.085921,103.359375,-280.69504,122.728615,...,2.753912,0.057281,1.352473,1.512109,-2.217835,0.780849,-3.965543,-9.139271,-6.930955,classical


In [3]:
data.shape

(1000, 29)

In [4]:
# eliminamos la columna "filename" que no es necesaria
data = data.drop(['filename'], axis=1)

In [5]:
# codificamos las etiquetas
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,:-1], dtype=float))

In [7]:
# Dividimos los conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)

X_train shape (800, 27)
X_test shape (200, 27)
y_train shape (800,)
y_test shape (200,)


In [8]:
# utilizamos una red neuronal para la clasificación
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

nn = MLPClassifier(solver='adam', activation='relu', max_iter=3000, random_state=42)
nn.fit(X_train, y_train)
preds = nn.predict(X_test)

print('Accuracy Neural Network:', accuracy_score(y_test, preds))

Accuracy Neural Network: 0.625


# Optimización de hyper parámetros utilizando **GridSearchCV**


In [9]:
from sklearn.model_selection import GridSearchCV

parametros = {'solver': ['lbfgs', 'adam'],
              'max_iter': [1500, 2000, 2500, 3000],
              'alpha': 10.0 ** -np.arange(1, 10),
              'hidden_layer_sizes': [(5000, 10), (3000, 50), (100, 100), (50,50,50)],
              'random_state': [0, 1, 5, 25, 42]}

clf = GridSearchCV(MLPClassifier(), parametros, n_jobs=-1)
clf.fit(X_train, y_train)

KeyboardInterrupt: 